Part 1

In [45]:
# Import Libs

from bs4 import BeautifulSoup
import requests
import pandas as pd
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
import math
print('Installed Libraries')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Installed Libraries


In [2]:
# Scrape Data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scraped_data = BeautifulSoup(requests.get(url).text, 'lxml')

In [3]:
# Turn scraped data table into primitive Dataframe

cols = ['PostalCode', 'Borough', 'Neighbourhood']
neighbour_df = pd.DataFrame(columns=cols)

j = 0
for i in scraped_data.find('table').find_all('tr'):
    if (len(i.find_all('td')) > 0):
        PC = i.find_all('td')[0].text.strip()
        B = i.find_all('td')[1].text.strip()
        N = i.find_all('td')[2].text.strip()
        neighbour_df.loc[j] = [i.find_all('td')[0].text.strip(), i.find_all('td')[1].text.strip(), i.find_all('td')[2].text.strip()]
        j = j + 1
neighbour_df

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
# Clean Dataframe

neighbour_df = neighbour_df[neighbour_df.Borough != "Not assigned"].reset_index(drop=True)
neighbour_df = neighbour_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
neighbour_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']
for i, j in neighbour_df.iterrows():
    if j['Neighbourhood'] == 'Not assigned':
        j['Neighbourhood'] = j['Borough']
print(neighbour_df)
neighbour_df.shape

    PostalCode      Borough                                      Neighbourhood
0          M1B  Scarborough                                     Malvern, Rouge
1          M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V    Etobicoke  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


(103, 3)

Part 2

In [5]:
#Load Latitude and Longitude data 
gd = pd.read_csv('http://cocl.us/Geospatial_data')

In [6]:
# Merge with Neighbourhood Data
gd.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_neighbour_df = pd.merge(neighbour_df, gd, on="PostalCode", how='left')
geo_neighbour_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Part 3

In [7]:
geopy.geocoders.options.default_timeout = None

address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

D:\Softwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [8]:
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_neighbour_df['Latitude'], geo_neighbour_df['Longitude'], geo_neighbour_df['Borough'], geo_neighbour_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#1aff66',
        fill_opacity=0.5,
        parse_html=False).add_to(map)  
    
map

Since I don't know much about toronto, I just searched for famous areas and one of those was York.

In [9]:
york_data = geo_neighbour_df[geo_neighbour_df['Borough'].str.contains('York')].reset_index(drop=True)
york_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


Let's look at North York 

In [10]:
address = 'North York'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

D:\Softwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [11]:
# create map of Manhattan using latitude and longitude values
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

In [12]:
CLIENT_ID = 'BRXRTBE1IEXJYVUED44XMZK51AWXU2MYQD1YCCTRXS2T0OHS' # your Foursquare ID
CLIENT_SECRET = 'GTBNTSB1ZOBMGS4U1Q1EHZSLZLBHOQNYVTEMMSRG50PZ1ODB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRXRTBE1IEXJYVUED44XMZK51AWXU2MYQD1YCCTRXS2T0OHS
CLIENT_SECRET:GTBNTSB1ZOBMGS4U1Q1EHZSLZLBHOQNYVTEMMSRG50PZ1ODB


Exploring the first Neighbourhood

In [13]:
york_data.loc[0, 'Neighbourhood']

'Hillcrest Village'

Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


Now, let's get the top 100 venues that are in North York within a radius of 500 meters.

In [15]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=BRXRTBE1IEXJYVUED44XMZK51AWXU2MYQD1YCCTRXS2T0OHS&client_secret=GTBNTSB1ZOBMGS4U1Q1EHZSLZLBHOQNYVTEMMSRG50PZ1ODB&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebfc332b57e88001c31551a'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

Now going through the same processes as in the lab

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

D:\Softwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
North Park, Maple Leaf Park, Upwood Park
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Runnymede, The Junction North
Humber Summit
Humberlea, Emery
Weston


Let's check the size of the resulting dataframe

In [22]:
print(york_venues.shape)
york_venues.head()

(335, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


Let's check how many venues were returned for each neighborhood

In [23]:
york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Don Mills,25,25,25,25,25,25
Downsview,16,16,16,16,16,16
East Toronto,4,4,4,4,4,4
"Fairview, Henry Farm, Oriole",69,69,69,69,69,69


Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 115 uniques categories.


Analyzing each neighbourhood.

In [25]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
york_onehot.shape

york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.05,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.043478,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Caledonia-Fairbanks,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.250000,0.000000
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Don Mills,0.000000,0.0000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Downsview,0.000000,0.0625,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,East Toronto,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,"Fairview, Henry Farm, Oriole",0.000000,0.0000,0.014493,0.000000,0.014493,0.000000,0.000000,0.028986,0.028986,...,0.014493,0.014493,0.000000,0.00,0.014493,0.00,0.000000,0.000000,0.028986,0.000000
9,Glencairn,0.000000,0.0000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [27]:
york_grouped.shape

(27, 116)

Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.10
1          Coffee Shop  0.10
2  Fried Chicken Joint  0.05
3       Sandwich Place  0.05
4           Restaurant  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.09
1              Coffee Shop  0.09
2           Sandwich Place  0.09
3  Comfort Food Restaurant  0.04
4                  Butcher  0.04


----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1           Pool  0.25
2  Women's Store  0.25
3      Juice Bar  0.00
4       Pharmacy  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                venue  freq
0          Restaurant  0.25
1  Turkish Restaurant  0.25
2      Sandwich P

A function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Supermarket,Restaurant,Bridal Shop,Sandwich Place,Ice Cream Shop,Shopping Mall,Diner,Pizza Place
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Distribution Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Butcher,Grocery Store,Liquor Store,Café,Juice Bar,Pharmacy,Pizza Place
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Turkish Restaurant,Restaurant,Sandwich Place,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


Run *k*-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 2, 3, 3, 3, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york_data

york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Bus Station,Cosmetics Shop,Bank,Women's Store,Bakery,Food Court
2,M2K,North York,Bayview Village,43.786947,-79.385975,3.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Distribution Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's visualize the resulting clusters

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (math.isnan(cluster)):
        cluster = 1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine clusters

In [50]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

#york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]
#york_merged.loc[york_merged['Cluster Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,1.0,Park,Construction & Landscaping,Convenience Store,Bank,Yoga Studio,Distribution Center,Concert Hall,Cosmetics Shop,Curling Ice,Deli / Bodega
8,North York,1.0,Park,Construction & Landscaping,Food & Drink Shop,Yoga Studio,Discount Store,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Curling Ice
28,North York,1.0,Park,Construction & Landscaping,Bakery,Yoga Studio,Distribution Center,Concert Hall,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega


In [ ]:
#york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]